# Task 4.G - Training Model with Ditto (FAIR-DA4ER)

This notebook trains a deep learning Entity Resolution model using the Ditto framework from the FAIR-DA4ER repository.

**Ditto** uses pre-trained language models (like DistilBERT, RoBERTa) for entity matching. It treats the problem as a sequence pair classification task.

## References:
- Li et al., "Deep Entity Matching with Pre-Trained Language Models"
- GitHub: https://github.com/MarcoNapoleone/FAIR-DA4ER

## 1. Install Dependencies

In [44]:
# Install PyTorch with CUDA 12.8 support (for GPU training)
# First uninstall CPU version, then install CUDA version
!pip uninstall torch torchvision torchaudio -y
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

# Install other Ditto requirements
!pip install transformers tqdm jsonlines nltk tensorboardX scikit-learn pandas numpy scipy

Found existing installation: torch 2.10.0+cu128
Uninstalling torch-2.10.0+cu128:
  Successfully uninstalled torch-2.10.0+cu128
Found existing installation: torchvision 0.25.0+cu128
Uninstalling torchvision-0.25.0+cu128:
  Successfully uninstalled torchvision-0.25.0+cu128
Found existing installation: torchaudio 2.10.0+cu128
Uninstalling torchaudio-2.10.0+cu128:
  Successfully uninstalled torchaudio-2.10.0+cu128


You can safely remove it manually.
You can safely remove it manually.


Looking in indexes: https://download.pytorch.org/whl/cu128
  Using cached https://download.pytorch.org/whl/cu128/torch-2.10.0%2Bcu128-cp311-cp311-win_amd64.whl.metadata (29 kB)
  Using cached https://download.pytorch.org/whl/cu128/torchvision-0.25.0%2Bcu128-cp311-cp311-win_amd64.whl.metadata (5.5 kB)
  Using cached https://download.pytorch.org/whl/cu128/torchaudio-2.10.0%2Bcu128-cp311-cp311-win_amd64.whl.metadata (7.1 kB)
Using cached https://download.pytorch.org/whl/cu128/torch-2.10.0%2Bcu128-cp311-cp311-win_amd64.whl (2867.4 MB)
Using cached https://download.pytorch.org/whl/cu128/torchvision-0.25.0%2Bcu128-cp311-cp311-win_amd64.whl (9.4 MB)
Using cached https://download.pytorch.org/whl/cu128/torchaudio-2.10.0%2Bcu128-cp311-cp311-win_amd64.whl (2.0 MB)

   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   --------

In [2]:
import pandas as pd
import numpy as np
import os
import sys
import json
import time
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

# Paths
WORKSPACE = r'c:\Users\migli\HW6ID'
GROUND_TRUTH_DIR = os.path.join(WORKSPACE, 'ground_truth')
DITTO_DIR = os.path.join(WORKSPACE, 'FAIR-DA4ER', 'ditto')
BLOCKING_RESULTS_DIR = os.path.join(WORKSPACE, 'blocking_results')

# Create output directory for our dataset
DITTO_DATA_DIR = os.path.join(DITTO_DIR, 'data', 'cars')
os.makedirs(DITTO_DATA_DIR, exist_ok=True)

# Create results directory
DITTO_RESULTS_DIR = os.path.join(WORKSPACE, 'ditto_results')
os.makedirs(DITTO_RESULTS_DIR, exist_ok=True)

print(f"DITTO_DIR: {DITTO_DIR}")
print(f"DITTO_DATA_DIR: {DITTO_DATA_DIR}")
print(f"DITTO_RESULTS_DIR: {DITTO_RESULTS_DIR}")

DITTO_DIR: c:\Users\migli\HW6ID\FAIR-DA4ER\ditto
DITTO_DATA_DIR: c:\Users\migli\HW6ID\FAIR-DA4ER\ditto\data\cars
DITTO_RESULTS_DIR: c:\Users\migli\HW6ID\ditto_results


## 2. Load Source Data

In [2]:
# Load datasets A and B
dataset_A = pd.read_csv(os.path.join(GROUND_TRUTH_DIR, 'dataset_A_no_vin.csv'))
dataset_B = pd.read_csv(os.path.join(GROUND_TRUTH_DIR, 'dataset_B_no_vin.csv'))

print(f"Dataset A: {len(dataset_A)} records")
print(f"Dataset B: {len(dataset_B)} records")
print(f"\nDataset A columns: {list(dataset_A.columns)}")
print(f"Dataset B columns: {list(dataset_B.columns)}")

Dataset A: 14492 records
Dataset B: 15375 records

Dataset A columns: ['body_type', 'cylinders', 'description', 'drive_type', 'exterior_color', 'fuel_type', 'image_url', 'latitude', 'listing_date', 'location', 'longitude', 'manufacturer', 'mileage', 'model', 'price', 'source_dataset', 'transmission', 'vehicle_id', 'year']
Dataset B columns: ['body_type', 'cylinders', 'description', 'drive_type', 'exterior_color', 'fuel_type', 'image_url', 'latitude', 'listing_date', 'location', 'longitude', 'manufacturer', 'mileage', 'model', 'price', 'source_dataset', 'transmission', 'vehicle_id', 'year']


In [3]:
# Load train/validation/test sets
train_df = pd.read_csv(os.path.join(GROUND_TRUTH_DIR, 'train.csv'))
valid_df = pd.read_csv(os.path.join(GROUND_TRUTH_DIR, 'validation.csv'))
test_df = pd.read_csv(os.path.join(GROUND_TRUTH_DIR, 'test.csv'))

print(f"Train: {len(train_df)} pairs, Positives: {train_df['label'].sum()}")
print(f"Validation: {len(valid_df)} pairs, Positives: {valid_df['label'].sum()}")
print(f"Test: {len(test_df)} pairs, Positives: {test_df['label'].sum()}")

Train: 10788 pairs, Positives: 2724
Validation: 2311 pairs, Positives: 548
Test: 2313 pairs, Positives: 581


In [4]:
# Check train column structure
print("Train columns:")
print(train_df.columns.tolist())
print("\nSample row:")
print(train_df.head(2))

Train columns:
['vin', 'craigslist_id', 'used_cars_id', 'year', 'manufacturer_cr', 'manufacturer_uc', 'model_cr', 'model_uc', 'price_cr', 'price_uc', 'mileage_cr', 'mileage_uc', 'warnings', 'warning_details', 'label']

Sample row:
   vin  craigslist_id  used_cars_id  year manufacturer_cr manufacturer_uc  \
0  NaN     7316716231     267497466   NaN       Chevrolet       Chevrolet   
1  NaN     7307279962     272550118   NaN       Chevrolet           Mazda   

               model_cr model_uc  price_cr  price_uc  mileage_cr  mileage_uc  \
0  Colorado Z71 4X4 Gas    Cruze     39999   19990.0     13303.0         4.0   
1              Suburban   Mazda6     23999   33590.0    200754.0         3.0   

   warnings warning_details  label  
0       NaN             NaN      0  
1       NaN             NaN      0  


## 3. Convert to Ditto Serialization Format

Ditto expects data in the format:
```
COL attr1 VAL value1 COL attr2 VAL value2 ...\tCOL attr1 VAL value1 COL attr2 VAL value2 ...\tlabel
```

Where each line contains:
- Record 1 serialized with COL/VAL markers
- TAB separator
- Record 2 serialized with COL/VAL markers
- TAB separator
- Label (0 or 1)

In [5]:
def serialize_record(row, prefix_cr=True):
    """
    Serialize a record to Ditto format.
    Uses the columns from train.csv which have _cr (Craigslist) and _uc (Used Cars) suffixes.
    """
    tokens = []
    
    # Common attributes to serialize for matching
    if prefix_cr:
        # Craigslist record
        attrs = [
            ('year', 'year'),  # Use common year column
            ('manufacturer', 'manufacturer_cr'),
            ('model', 'model_cr'),
            ('price', 'price_cr'),
            ('mileage', 'mileage_cr'),
        ]
    else:
        # Used Cars record
        attrs = [
            ('year', 'year'),  # Use common year column
            ('manufacturer', 'manufacturer_uc'),
            ('model', 'model_uc'),
            ('price', 'price_uc'),
            ('mileage', 'mileage_uc'),
        ]
    
    for attr_name, col_name in attrs:
        if col_name in row.index:
            val = row[col_name]
            if pd.notna(val):
                val_str = str(val).strip()
                if val_str:
                    tokens.append(f"COL {attr_name} VAL {val_str}")
    
    return ' '.join(tokens)


def convert_to_ditto_format(df):
    """
    Convert a DataFrame with paired records to Ditto format.
    """
    lines = []
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Converting to Ditto format"):
        rec1 = serialize_record(row, prefix_cr=True)
        rec2 = serialize_record(row, prefix_cr=False)
        label = int(row['label'])
        
        line = f"{rec1}\t{rec2}\t{label}"
        lines.append(line)
    
    return lines

# Test on a sample
sample_row = train_df.iloc[5]  # A positive match
print("Sample row (label={}):".format(sample_row['label']))
print(serialize_record(sample_row, prefix_cr=True))
print("---")
print(serialize_record(sample_row, prefix_cr=False))

Sample row (label=0):
COL manufacturer VAL Toyota COL model VAL Avalon Limited Sedan 4D COL price VAL 37590 COL mileage VAL 3939.0
---
COL manufacturer VAL Toyota COL model VAL Corolla Hybrid COL price VAL 25183.0 COL mileage VAL 3.0


In [6]:
# Convert all datasets to Ditto format
print("Converting train set...")
train_lines = convert_to_ditto_format(train_df)

print("\nConverting validation set...")
valid_lines = convert_to_ditto_format(valid_df)

print("\nConverting test set...")
test_lines = convert_to_ditto_format(test_df)

print(f"\nConverted: train={len(train_lines)}, valid={len(valid_lines)}, test={len(test_lines)}")

Converting train set...


Converting to Ditto format: 100%|██████████| 10788/10788 [00:00<00:00, 44559.97it/s]



Converting validation set...


Converting to Ditto format: 100%|██████████| 2311/2311 [00:00<00:00, 46220.48it/s]



Converting test set...


Converting to Ditto format: 100%|██████████| 2313/2313 [00:00<00:00, 46260.71it/s]


Converted: train=10788, valid=2311, test=2313


In [7]:
# Save to files
train_path = os.path.join(DITTO_DATA_DIR, 'train.txt')
valid_path = os.path.join(DITTO_DATA_DIR, 'valid.txt')
test_path = os.path.join(DITTO_DATA_DIR, 'test.txt')

with open(train_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(train_lines))

with open(valid_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(valid_lines))

with open(test_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(test_lines))

print(f"Saved train.txt: {train_path}")
print(f"Saved valid.txt: {valid_path}")
print(f"Saved test.txt: {test_path}")

# Show sample
print("\nSample lines from train.txt:")
print(train_lines[0])
print("---")
print(train_lines[5])  # A positive example

Saved train.txt: c:\Users\migli\HW6ID\FAIR-DA4ER\ditto\data\cars\train.txt
Saved valid.txt: c:\Users\migli\HW6ID\FAIR-DA4ER\ditto\data\cars\valid.txt
Saved test.txt: c:\Users\migli\HW6ID\FAIR-DA4ER\ditto\data\cars\test.txt

Sample lines from train.txt:
COL manufacturer VAL Chevrolet COL model VAL Colorado Z71 4X4 Gas COL price VAL 39999 COL mileage VAL 13303.0	COL manufacturer VAL Chevrolet COL model VAL Cruze COL price VAL 19990.0 COL mileage VAL 4.0	0
---
COL manufacturer VAL Toyota COL model VAL Avalon Limited Sedan 4D COL price VAL 37590 COL mileage VAL 3939.0	COL manufacturer VAL Toyota COL model VAL Corolla Hybrid COL price VAL 25183.0 COL mileage VAL 3.0	0


## 4. Add Configuration to configs.json

In [8]:
# Load existing configs and add our task
configs_path = os.path.join(DITTO_DIR, 'configs.json')

with open(configs_path, 'r') as f:
    configs = json.load(f)

# Check if our config already exists
our_config = {
    "name": "Cars_ER",
    "task_type": "classification",
    "vocab": ["0", "1"],
    "trainset": "data/cars/train.txt",
    "validset": "data/cars/valid.txt",
    "testset": "data/cars/test.txt"
}

# Add if not exists
config_names = [c['name'] for c in configs]
if 'Cars_ER' not in config_names:
    configs.append(our_config)
    with open(configs_path, 'w') as f:
        json.dump(configs, f, indent=2)
    print("Added Cars_ER configuration to configs.json")
else:
    print("Cars_ER configuration already exists")

print(f"\nOur config: {our_config}")

Cars_ER configuration already exists

Our config: {'name': 'Cars_ER', 'task_type': 'classification', 'vocab': ['0', '1'], 'trainset': 'data/cars/train.txt', 'validset': 'data/cars/valid.txt', 'testset': 'data/cars/test.txt'}


## 5. Train Ditto Model

In [3]:
# Change to ditto directory and add to path
os.chdir(DITTO_DIR)
sys.path.insert(0, DITTO_DIR)
print(f"Working directory: {os.getcwd()}")

Working directory: c:\Users\migli\HW6ID\FAIR-DA4ER\ditto


In [10]:
# Download NLTK stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\migli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\migli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\migli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
# Import Ditto components
import torch
import random

# Set seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
    
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.10.0+cu128
CUDA available: True
CUDA device: NVIDIA GeForce RTX 5080


In [13]:
# Import Ditto modules
from ditto_light.dataset import DittoDataset
from ditto_light.ditto import train as ditto_train, evaluate, DittoModel

print("Ditto modules imported successfully")

c:\Users\migli\HW6ID\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Ditto modules imported successfully


In [15]:
# Load configs
os.chdir(DITTO_DIR)
configs_path = os.path.join(DITTO_DIR, 'configs.json')
with open(configs_path, 'r') as f:
    configs = json.load(f)
configs_dict = {c['name']: c for c in configs}
config = configs_dict['Cars_ER']

print(f"Config: {config}")


Config: {'name': 'Cars_ER', 'task_type': 'classification', 'vocab': ['0', '1'], 'trainset': 'data/cars/train.txt', 'validset': 'data/cars/valid.txt', 'testset': 'data/cars/test.txt'}


In [5]:
# Hyperparameters
class HyperParams:
    def __init__(self):
        self.task = 'Cars_ER'
        self.batch_size = 32
        self.max_len = 128
        self.lr = 3e-5
        self.n_epochs = 10
        self.lm = 'distilbert'  # Faster than RoBERTa
        self.da = None  # Data augmentation
        self.dk = None  # Domain knowledge
        self.summarize = False
        self.alpha_aug = 0.8
        self.run_id = 0
        self.save_model = True
        self.logdir = DITTO_RESULTS_DIR
        # Force GPU usage
        self.device = 'cuda'
        self.fp16 = True  # Enable mixed precision for faster GPU training

hp = HyperParams()

# Verify GPU availability
if not torch.cuda.is_available():
    raise RuntimeError("CUDA GPU not available! This training requires a GPU.")
    
print(f"✅ Device: {hp.device}")
print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
print(f"✅ FP16 (Mixed Precision): {hp.fp16}")
print(f"LM: {hp.lm}")
print(f"Epochs: {hp.n_epochs}")
print(f"Batch size: {hp.batch_size}")

✅ Device: cuda
✅ GPU: NVIDIA GeForce RTX 5080
✅ FP16 (Mixed Precision): True
LM: distilbert
Epochs: 10
Batch size: 32


In [16]:
# Load datasets
print("Loading train dataset...")
train_dataset = DittoDataset(config['trainset'], lm=hp.lm, max_len=hp.max_len, da=hp.da)
print(f"Train dataset: {len(train_dataset)} examples")

print("\nLoading validation dataset...")
valid_dataset = DittoDataset(config['validset'], lm=hp.lm, max_len=hp.max_len)
print(f"Validation dataset: {len(valid_dataset)} examples")

print("\nLoading test dataset...")
test_dataset = DittoDataset(config['testset'], lm=hp.lm, max_len=hp.max_len)
print(f"Test dataset: {len(test_dataset)} examples")

Loading train dataset...
Train dataset: 10788 examples

Loading validation dataset...
Validation dataset: 2311 examples

Loading test dataset...
Test dataset: 2313 examples


In [17]:
# Train the model
print("="*60)
print("TRAINING DITTO MODEL")
print("="*60)

run_tag = f"{hp.task}_lm={hp.lm}_epochs={hp.n_epochs}"

training_start = time.time()

ditto_train(train_dataset, valid_dataset, test_dataset, run_tag, hp)

training_time = time.time() - training_start
print(f"\nTraining completed in {training_time:.2f} seconds")

TRAINING DITTO MODEL
Running cuda


TRAINING DITTO MODEL
Running cuda


Loading weights: 100%|██████████| 100/100 [00:00<00:00, 1389.76it/s, Materializing param=transformer.layer.5.sa_layer_norm.weight]   
DistilBertModel LOAD REPORT from: distilbert-base-uncased
Key                     | Status     |  | 
------------------------+------------+--+-
vocab_layer_norm.weight | UNEXPECTED |  | 
vocab_projector.bias    | UNEXPECTED |  | 
vocab_layer_norm.bias   | UNEXPECTED |  | 
vocab_transform.bias    | UNEXPECTED |  | 
vocab_transform.weight  | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
c:\Users\migli\HW6ID\FAIR-DA4ER\ditto\ditto_light\ditto.py:144: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at htt

TRAINING DITTO MODEL
Running cuda


Loading weights: 100%|██████████| 100/100 [00:00<00:00, 1389.76it/s, Materializing param=transformer.layer.5.sa_layer_norm.weight]   
DistilBertModel LOAD REPORT from: distilbert-base-uncased
Key                     | Status     |  | 
------------------------+------------+--+-
vocab_layer_norm.weight | UNEXPECTED |  | 
vocab_projector.bias    | UNEXPECTED |  | 
vocab_layer_norm.bias   | UNEXPECTED |  | 
vocab_transform.bias    | UNEXPECTED |  | 
vocab_transform.weight  | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
c:\Users\migli\HW6ID\FAIR-DA4ER\ditto\ditto_light\ditto.py:144: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at htt

step: 0, loss: 0.657928466796875
step: 10, loss: 0.5758132934570312
step: 20, loss: 0.4860572814941406
step: 30, loss: 0.13100528717041016
step: 40, loss: 0.5732955932617188
step: 50, loss: 0.03508177399635315
step: 60, loss: 0.1079677939414978
step: 70, loss: 0.021427392959594727
step: 80, loss: 0.0054675862193107605
step: 90, loss: 0.2797318398952484
step: 100, loss: 0.10306283831596375
step: 110, loss: 0.028403908014297485
step: 120, loss: 0.20374122262001038
step: 130, loss: 0.010962992906570435
step: 140, loss: 0.011815749108791351
step: 150, loss: 0.005495689809322357
step: 160, loss: 0.005922652781009674
step: 170, loss: 0.13485746085643768
step: 180, loss: 0.0963040292263031
step: 190, loss: 0.004587128758430481
step: 200, loss: 0.010635815560817719
step: 210, loss: 0.014300454407930374
step: 220, loss: 0.01826491579413414
step: 230, loss: 0.15486572682857513
step: 240, loss: 0.01771315187215805
step: 250, loss: 0.005220070481300354
step: 260, loss: 0.20614036917686462
step: 27

## 6. Load Trained Model and Evaluate

In [18]:
# Find the saved model
model_dir = os.path.join(DITTO_RESULTS_DIR, hp.task)  # Model saved under task name
model_path = os.path.join(model_dir, 'model.pt')

if os.path.exists(model_path):
    print(f"Found trained model: {model_path}")
else:
    # Try alternative path with run_tag
    model_dir = os.path.join(DITTO_RESULTS_DIR, run_tag.replace('/', '_'))
    model_path = os.path.join(model_dir, 'model.pt')
    if os.path.exists(model_path):
        print(f"Found trained model: {model_path}")
    else:
        # List what we have
        print(f"Looking for model in: {DITTO_RESULTS_DIR}")
        for root, dirs, files in os.walk(DITTO_RESULTS_DIR):
            for f in files:
                if f.endswith('.pt'):
                    model_path = os.path.join(root, f)
                    print(f"Found: {model_path}")

Found trained model: c:\Users\migli\HW6ID\ditto_results\Cars_ER\model.pt


In [19]:
# Load the model for inference
from torch.utils import data

def load_model_for_inference(model_path, device, lm='distilbert'):
    """Load trained Ditto model for inference."""
    model = DittoModel(device=device, lm=lm)
    
    # Load checkpoint
    checkpoint = torch.load(model_path, map_location=device, weights_only=False)
    
    # Check if it's a full checkpoint or just state_dict
    if isinstance(checkpoint, dict) and 'model' in checkpoint:
        # Full checkpoint format
        state_dict = checkpoint['model']
    else:
        # Direct state_dict
        state_dict = checkpoint
    
    model.load_state_dict(state_dict)
    
    if device == 'cuda':
        model = model.cuda()
    model.eval()
    
    return model

# Load model
model = load_model_for_inference(model_path, hp.device, hp.lm)
print("Model loaded successfully")

Loading weights: 100%|██████████| 100/100 [00:00<00:00, 956.83it/s, Materializing param=transformer.layer.5.sa_layer_norm.weight]   
DistilBertModel LOAD REPORT from: distilbert-base-uncased
Key                     | Status     |  | 
------------------------+------------+--+-
vocab_layer_norm.weight | UNEXPECTED |  | 
vocab_projector.bias    | UNEXPECTED |  | 
vocab_layer_norm.bias   | UNEXPECTED |  | 
vocab_transform.bias    | UNEXPECTED |  | 
vocab_transform.weight  | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Loading weights: 100%|██████████| 100/100 [00:00<00:00, 956.83it/s, Materializing param=transformer.layer.5.sa_layer_norm.weight]   
DistilBertModel LOAD REPORT from: distilbert-base-uncased
Key                     | Status     |  | 
------------------------+------------+--+-
vocab_layer_norm.weight | UNEXPECTED |  | 
vocab_projector.bias    | UNEXPECTED |  | 
vocab_layer_norm.bias   | UNEXPECTED |  | 
vocab_transform.bias    | UNEXPECTED |  | 
vocab_transform.weight  | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded successfully


## 7. Prepare Blocking Candidates for Evaluation

In [20]:
# Load blocking candidates
B1_path = os.path.join(BLOCKING_RESULTS_DIR, 'candidates_B1.csv')
B2_path = os.path.join(BLOCKING_RESULTS_DIR, 'candidates_B2.csv')

B1_candidates = pd.read_csv(B1_path)
B2_candidates = pd.read_csv(B2_path)

print(f"B1 candidates: {len(B1_candidates)} pairs")
print(f"B2 candidates: {len(B2_candidates)} pairs")

B1 candidates: 964002 pairs
B2 candidates: 77087 pairs


In [21]:
# Load ground truth for evaluation
ground_truth = pd.read_csv(os.path.join(GROUND_TRUTH_DIR, 'ground_truth_complete.csv'))
print(f"Ground truth: {len(ground_truth)} matches")

# Create ground truth set using craigslist_id and used_cars_id
# These match the vehicle_id in the original datasets which are used as id_A and id_B in blocking
gt_set = set(zip(ground_truth['craigslist_id'].astype(str), ground_truth['used_cars_id'].astype(str)))
print(f"Ground truth set size: {len(gt_set)}")

Ground truth: 15412 matches
Ground truth set size: 15412


In [23]:
# Function to create Ditto-format pairs from blocking candidates
def prepare_blocking_for_ditto(candidates_df, dataset_A, dataset_B):
    """
    Create Ditto-format pairs from blocking candidates.
    """
    # Ensure IDs are strings for matching
    dataset_A = dataset_A.copy()
    dataset_B = dataset_B.copy()
    
    # Get ID column names - use vehicle_id for our datasets
    A_id_col = 'vehicle_id'
    B_id_col = 'vehicle_id'
    
    dataset_A[A_id_col] = dataset_A[A_id_col].astype(str)
    dataset_B[B_id_col] = dataset_B[B_id_col].astype(str)
    
    # Create lookup dictionaries
    A_dict = dataset_A.set_index(A_id_col).to_dict('index')
    B_dict = dataset_B.set_index(B_id_col).to_dict('index')
    
    lines = []
    pair_ids = []
    
    # Get candidate ID columns - blocking files use id_A and id_B
    cand_A_col = 'id_A'
    cand_B_col = 'id_B'
    
    for _, row in tqdm(candidates_df.iterrows(), total=len(candidates_df), desc="Preparing candidates"):
        id_A = str(row[cand_A_col])
        id_B = str(row[cand_B_col])
        
        if id_A not in A_dict or id_B not in B_dict:
            continue
        
        rec_A = A_dict[id_A]
        rec_B = B_dict[id_B]
        
        # Serialize records
        tokens_A = []
        tokens_B = []
        
        # Map A (Craigslist) attributes
        A_mapping = {
            'year': 'year',
            'manufacturer': 'manufacturer',
            'model': 'model',
            'price': 'price',
            'mileage': 'mileage',
            'fuel_type': 'fuel',
            'transmission': 'transmission'
        }
        
        for attr, ditto_attr in A_mapping.items():
            if attr in rec_A and pd.notna(rec_A[attr]):
                val = str(rec_A[attr]).strip()
                if val:
                    tokens_A.append(f"COL {ditto_attr} VAL {val}")
        
        # Map B (Used Cars) attributes - same structure
        B_mapping = {
            'year': 'year',
            'manufacturer': 'manufacturer',
            'model': 'model',
            'price': 'price',
            'mileage': 'mileage',
            'fuel_type': 'fuel',
            'transmission': 'transmission'
        }
        
        for attr, ditto_attr in B_mapping.items():
            if attr in rec_B and pd.notna(rec_B[attr]):
                val = str(rec_B[attr]).strip()
                if val:
                    tokens_B.append(f"COL {ditto_attr} VAL {val}")
        
        rec1_str = ' '.join(tokens_A)
        rec2_str = ' '.join(tokens_B)
        
        # Use dummy label 0, will be replaced with prediction
        line = f"{rec1_str}\t{rec2_str}\t0"
        lines.append(line)
        pair_ids.append((id_A, id_B))
    
    return lines, pair_ids

print("Function defined")

Function defined


In [24]:
# Reload datasets A and B from original
os.chdir(WORKSPACE)
dataset_A = pd.read_csv(os.path.join(GROUND_TRUTH_DIR, 'dataset_A_no_vin.csv'))
dataset_B = pd.read_csv(os.path.join(GROUND_TRUTH_DIR, 'dataset_B_no_vin.csv'))
os.chdir(DITTO_DIR)

print(f"Dataset A columns: {list(dataset_A.columns)[:10]}")
print(f"Dataset B columns: {list(dataset_B.columns)[:10]}")

Dataset A columns: ['body_type', 'cylinders', 'description', 'drive_type', 'exterior_color', 'fuel_type', 'image_url', 'latitude', 'listing_date', 'location']
Dataset B columns: ['body_type', 'cylinders', 'description', 'drive_type', 'exterior_color', 'fuel_type', 'image_url', 'latitude', 'listing_date', 'location']


In [25]:
# Prepare B1 candidates
print("Preparing B1 candidates...")
B1_lines, B1_pair_ids = prepare_blocking_for_ditto(B1_candidates, dataset_A, dataset_B)
print(f"B1: {len(B1_lines)} pairs prepared")

print("\nPreparing B2 candidates...")
B2_lines, B2_pair_ids = prepare_blocking_for_ditto(B2_candidates, dataset_A, dataset_B)
print(f"B2: {len(B2_lines)} pairs prepared")

Preparing B1 candidates...


Preparing candidates: 100%|██████████| 964002/964002 [00:14<00:00, 64875.82it/s]


B1: 964002 pairs prepared

Preparing B2 candidates...


Preparing candidates: 100%|██████████| 77087/77087 [00:01<00:00, 64223.97it/s]

B2: 77087 pairs prepared


## 8. Run Inference and Evaluate

In [26]:
def predict_with_ditto(model, lines, batch_size=256, lm='distilbert', max_len=128, device='cuda'):
    """
    Run Ditto model on a list of serialized pairs.
    Returns probabilities for class 1 (match).
    """
    from torch.utils.data import DataLoader
    import torch.nn.functional as F
    
    # Create dataset from lines
    dataset = DittoDataset(lines, lm=lm, max_len=max_len)
    
    # Create dataloader
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, 
                        collate_fn=dataset.pad, num_workers=0)
    
    model.eval()
    all_probs = []
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Predicting"):
            if len(batch) == 2:
                x, y = batch
                if device == 'cuda':
                    x = x.cuda()
                logits = model(x)
            else:
                x1, x2, y = batch
                if device == 'cuda':
                    x1 = x1.cuda()
                    x2 = x2.cuda()
                logits = model(x1, x2)
            
            # Get probabilities
            probs = F.softmax(logits, dim=1)
            # Get probability of class 1 (match)
            match_probs = probs[:, 1].cpu().numpy()
            all_probs.extend(match_probs)
    
    return np.array(all_probs)

print("Prediction function defined")

Prediction function defined


In [27]:
def evaluate_predictions(predictions, pair_ids, gt_set, threshold=0.5):
    """
    Evaluate predictions against ground truth.
    """
    # Create predictions based on threshold
    pred_labels = (predictions >= threshold).astype(int)
    
    # Get true labels from ground truth
    true_labels = [1 if (id_A, id_B) in gt_set else 0 for id_A, id_B in pair_ids]
    true_labels = np.array(true_labels)
    
    # Calculate metrics
    precision = precision_score(true_labels, pred_labels, zero_division=0)
    recall = recall_score(true_labels, pred_labels, zero_division=0)
    f1 = f1_score(true_labels, pred_labels, zero_division=0)
    accuracy = accuracy_score(true_labels, pred_labels)
    
    # Confusion matrix
    tn, fp, fn, tp = confusion_matrix(true_labels, pred_labels, labels=[0, 1]).ravel()
    
    return {
        'threshold': threshold,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy,
        'tp': tp,
        'fp': fp,
        'tn': tn,
        'fn': fn,
        'total_positives_gt': int(sum(true_labels)),
        'predicted_positives': int(sum(pred_labels))
    }

print("Evaluation function defined")

Evaluation function defined


In [34]:
# Run inference on B1
print("="*60)
print("EVALUATING B1-Ditto")
print("="*60)

B1_inference_start = time.time()
B1_probs = predict_with_ditto(model, B1_lines, batch_size=256, lm=hp.lm, max_len=hp.max_len, device=hp.device)
B1_inference_time = time.time() - B1_inference_start

print(f"\nB1 Inference time: {B1_inference_time:.2f} seconds")
print(f"B1 Predictions: {len(B1_probs)}")

EVALUATING B1-Ditto


Predicting: 100%|██████████| 3766/3766 [06:44<00:00,  9.30it/s]


B1 Inference time: 406.29 seconds
B1 Predictions: 964002


In [35]:
# Evaluate B1 with multiple thresholds
thresholds = [0.3, 0.5, 0.7]
B1_results = []

for thresh in thresholds:
    metrics = evaluate_predictions(B1_probs, B1_pair_ids, gt_set, threshold=thresh)
    B1_results.append(metrics)
    print(f"\nB1 @ threshold={thresh}:")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall: {metrics['recall']:.4f}")
    print(f"  F1: {metrics['f1']:.4f}")
    print(f"  TP: {metrics['tp']}, FP: {metrics['fp']}, FN: {metrics['fn']}, TN: {metrics['tn']}")


B1 @ threshold=0.3:
  Precision: 0.0103
  Recall: 0.9271
  F1: 0.0204
  TP: 2186, FP: 210150, FN: 172, TN: 751494

B1 @ threshold=0.5:
  Precision: 0.0127
  Recall: 0.9075
  F1: 0.0250
  TP: 2140, FP: 166686, FN: 218, TN: 794958

B1 @ threshold=0.7:
  Precision: 0.0158
  Recall: 0.8838
  F1: 0.0311
  TP: 2084, FP: 129638, FN: 274, TN: 832006


In [36]:
# Run inference on B2
print("="*60)
print("EVALUATING B2-Ditto")
print("="*60)

B2_inference_start = time.time()
B2_probs = predict_with_ditto(model, B2_lines, batch_size=256, lm=hp.lm, max_len=hp.max_len, device=hp.device)
B2_inference_time = time.time() - B2_inference_start

print(f"\nB2 Inference time: {B2_inference_time:.2f} seconds")
print(f"B2 Predictions: {len(B2_probs)}")

EVALUATING B2-Ditto


Predicting: 100%|██████████| 302/302 [00:33<00:00,  9.04it/s]


B2 Inference time: 34.33 seconds
B2 Predictions: 77087


In [37]:
# Evaluate B2 with multiple thresholds
B2_results = []

for thresh in thresholds:
    metrics = evaluate_predictions(B2_probs, B2_pair_ids, gt_set, threshold=thresh)
    B2_results.append(metrics)
    print(f"\nB2 @ threshold={thresh}:")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall: {metrics['recall']:.4f}")
    print(f"  F1: {metrics['f1']:.4f}")
    print(f"  TP: {metrics['tp']}, FP: {metrics['fp']}, FN: {metrics['fn']}, TN: {metrics['tn']}")


B2 @ threshold=0.3:
  Precision: 0.0576
  Recall: 0.9424
  F1: 0.1086
  TP: 2177, FP: 35600, FN: 133, TN: 39177

B2 @ threshold=0.5:
  Precision: 0.0676
  Recall: 0.9238
  F1: 0.1259
  TP: 2134, FP: 29454, FN: 176, TN: 45323

B2 @ threshold=0.7:
  Precision: 0.0804
  Recall: 0.9009
  F1: 0.1476
  TP: 2081, FP: 23813, FN: 229, TN: 50964


In [32]:
# Debug: Check probability distribution
print("B1 probability distribution:")
print(f"  Min: {B1_probs.min():.4f}")
print(f"  Max: {B1_probs.max():.4f}")
print(f"  Mean: {B1_probs.mean():.4f}")
print(f"  Median: {np.median(B1_probs):.4f}")
print(f"  > 0.5: {(B1_probs > 0.5).sum()}")
print(f"  > 0.9: {(B1_probs > 0.9).sum()}")
print(f"  > 0.99: {(B1_probs > 0.99).sum()}")

print("\nB2 probability distribution:")
print(f"  Min: {B2_probs.min():.4f}")
print(f"  Max: {B2_probs.max():.4f}")
print(f"  Mean: {B2_probs.mean():.4f}")
print(f"  Median: {np.median(B2_probs):.4f}")
print(f"  > 0.5: {(B2_probs > 0.5).sum()}")
print(f"  > 0.9: {(B2_probs > 0.9).sum()}")
print(f"  > 0.99: {(B2_probs > 0.99).sum()}")

B1 probability distribution:
  Min: 0.0000
  Max: 0.9998
  Mean: 0.1891
  Median: 0.0098
  > 0.5: 168826
  > 0.9: 84415
  > 0.99: 29862

B2 probability distribution:
  Min: 0.0001
  Max: 0.9998
  Mean: 0.4197
  Median: 0.2785
  > 0.5: 31588
  > 0.9: 18149
  > 0.99: 8272


In [33]:
# Check training data distribution - are labels balanced?
print("Checking training data distribution...")
with open(train_path, 'r', encoding='utf-8') as f:
    train_labels = [int(line.strip().split('\t')[-1]) for line in f]
    
print(f"Train labels - 0 (non-match): {train_labels.count(0)}, 1 (match): {train_labels.count(1)}")

with open(valid_path, 'r', encoding='utf-8') as f:
    valid_labels = [int(line.strip().split('\t')[-1]) for line in f]
print(f"Valid labels - 0 (non-match): {valid_labels.count(0)}, 1 (match): {valid_labels.count(1)}")

with open(test_path, 'r', encoding='utf-8') as f:
    test_labels = [int(line.strip().split('\t')[-1]) for line in f]
print(f"Test labels - 0 (non-match): {test_labels.count(0)}, 1 (match): {test_labels.count(1)}")

# Also check a few samples
print("\n--- Sample training lines ---")
with open(train_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 5:
            break
        parts = line.strip().split('\t')
        print(f"Label: {parts[-1]}")

Checking training data distribution...


NameError: name 'train_path' is not defined

In [39]:
# Test model on test set to see if it's working
print("Testing model on test set...")

# Load test data
with open(test_path, 'r', encoding='utf-8') as f:
    test_all_lines = [line.strip() for line in f]

# Get predictions
test_preds = predict_with_ditto(model, test_all_lines, batch_size=256, lm=hp.lm, max_len=hp.max_len, device=hp.device)

# Get true labels
test_true = [int(line.split('\t')[-1]) for line in test_all_lines]

print(f"\nTest Set Predictions:")
print(f"  Min prob: {test_preds.min():.4f}")
print(f"  Max prob: {test_preds.max():.4f}")
print(f"  Mean prob: {test_preds.mean():.4f}")
print(f"  Median prob: {np.median(test_preds):.4f}")

# Distribution by true label
test_true = np.array(test_true)
print(f"\nFor true NON-MATCH (label=0): mean pred = {test_preds[test_true == 0].mean():.4f}")
print(f"For true MATCH (label=1): mean pred = {test_preds[test_true == 1].mean():.4f}")

# Evaluate on test set
from sklearn.metrics import classification_report
test_pred_labels = (test_preds >= 0.5).astype(int)
print("\nClassification Report on Test Set:")
print(classification_report(test_true, test_pred_labels, target_names=['Non-Match', 'Match']))

Testing model on test set...


Predicting: 100%|██████████| 10/10 [00:01<00:00,  9.49it/s]


Test Set Predictions:
  Min prob: 0.0000
  Max prob: 0.9999
  Mean prob: 0.2512
  Median prob: 0.0000

For true NON-MATCH (label=0): mean pred = 0.0000
For true MATCH (label=1): mean pred = 0.9999

Classification Report on Test Set:
              precision    recall  f1-score   support

   Non-Match       1.00      1.00      1.00      1732
       Match       1.00      1.00      1.00       581

    accuracy                           1.00      2313
   macro avg       1.00      1.00      1.00      2313
weighted avg       1.00      1.00      1.00      2313



In [40]:
# Check sample B1 lines vs sample test lines
print("=== Sample TEST line (working correctly) ===")
print(test_all_lines[0])
print()

print("=== Sample B1 lines (not working) ===")
for i in range(3):
    print(B1_lines[i])
print()

# Check if B1_lines have the right format
print("=== Checking if B1_lines look correct ===")
print(f"Number of TAB separators in test line: {test_all_lines[0].count(chr(9))}")
print(f"Number of TAB separators in B1 line: {B1_lines[0].count(chr(9))}")

=== Sample TEST line (working correctly) ===
COL manufacturer VAL Nissan COL model VAL Maxima COL price VAL 3975 COL mileage VAL 147799.0	COL manufacturer VAL Chevrolet COL model VAL Equinox COL price VAL 23934.0 COL mileage VAL 5.0	0

=== Sample B1 lines (not working) ===
COL year VAL 2014.0 COL manufacturer VAL Mazda COL model VAL Mazda5 COL price VAL 0 COL mileage VAL 60047.0 COL fuel VAL gas COL transmission VAL automatic	COL year VAL 2008 COL manufacturer VAL Mazda COL model VAL Mazda5 COL price VAL 4999.0 COL mileage VAL 130122.0 COL fuel VAL gasoline COL transmission VAL automatic	0
COL year VAL 2014.0 COL manufacturer VAL Mazda COL model VAL Mazda5 COL price VAL 0 COL mileage VAL 60047.0 COL fuel VAL gas COL transmission VAL automatic	COL year VAL 2012 COL manufacturer VAL Mazda COL model VAL Mazda5 COL price VAL 7999.0 COL mileage VAL 106803.0 COL fuel VAL gasoline COL transmission VAL 5-speed automatic overdrive	0
COL year VAL 2014.0 COL manufacturer VAL Mazda COL model VAL M

In [41]:
# FIXED: Recreate B1/B2 lines with the SAME format as training data
# Training uses: manufacturer, model, price, mileage (NO year, fuel, transmission)

def prepare_blocking_for_ditto_fixed(candidates_df, dataset_A, dataset_B):
    """
    Create Ditto-format pairs from blocking candidates.
    Using SAME format as training data.
    """
    dataset_A = dataset_A.copy()
    dataset_B = dataset_B.copy()
    
    A_id_col = 'vehicle_id'
    B_id_col = 'vehicle_id'
    
    dataset_A[A_id_col] = dataset_A[A_id_col].astype(str)
    dataset_B[B_id_col] = dataset_B[B_id_col].astype(str)
    
    A_dict = dataset_A.set_index(A_id_col).to_dict('index')
    B_dict = dataset_B.set_index(B_id_col).to_dict('index')
    
    lines = []
    pair_ids = []
    
    cand_A_col = 'id_A'
    cand_B_col = 'id_B'
    
    # Use SAME attributes as training: manufacturer, model, price, mileage
    # NOTE: Training used year, manufacturer, model, price, mileage
    attr_mapping = [
        ('year', 'year'),
        ('manufacturer', 'manufacturer'),
        ('model', 'model'),
        ('price', 'price'),
        ('mileage', 'mileage'),
    ]
    
    for _, row in tqdm(candidates_df.iterrows(), total=len(candidates_df), desc="Preparing candidates"):
        id_A = str(row[cand_A_col])
        id_B = str(row[cand_B_col])
        
        if id_A not in A_dict or id_B not in B_dict:
            continue
        
        rec_A = A_dict[id_A]
        rec_B = B_dict[id_B]
        
        tokens_A = []
        tokens_B = []
        
        for ditto_attr, data_attr in attr_mapping:
            if data_attr in rec_A and pd.notna(rec_A[data_attr]):
                val = str(rec_A[data_attr]).strip()
                if val:
                    tokens_A.append(f"COL {ditto_attr} VAL {val}")
                    
        for ditto_attr, data_attr in attr_mapping:
            if data_attr in rec_B and pd.notna(rec_B[data_attr]):
                val = str(rec_B[data_attr]).strip()
                if val:
                    tokens_B.append(f"COL {ditto_attr} VAL {val}")
        
        rec1_str = ' '.join(tokens_A)
        rec2_str = ' '.join(tokens_B)
        
        # Dummy label
        line = f"{rec1_str}\t{rec2_str}\t0"
        lines.append(line)
        pair_ids.append((id_A, id_B))
    
    return lines, pair_ids

print("Fixed function defined")

Fixed function defined


In [42]:
# Reload datasets and regenerate B1/B2 lines with fixed format
os.chdir(WORKSPACE)
dataset_A = pd.read_csv(os.path.join(GROUND_TRUTH_DIR, 'dataset_A_no_vin.csv'))
dataset_B = pd.read_csv(os.path.join(GROUND_TRUTH_DIR, 'dataset_B_no_vin.csv'))

# Reload blocking candidates
B1_candidates = pd.read_csv(os.path.join(BLOCKING_RESULTS_DIR, 'candidates_B1.csv'))
B2_candidates = pd.read_csv(os.path.join(BLOCKING_RESULTS_DIR, 'candidates_B2.csv'))

print(f"Dataset A columns: {list(dataset_A.columns)}")
print(f"Dataset B columns: {list(dataset_B.columns)}")
print(f"B1 candidates: {len(B1_candidates)}")
print(f"B2 candidates: {len(B2_candidates)}")

Dataset A columns: ['body_type', 'cylinders', 'description', 'drive_type', 'exterior_color', 'fuel_type', 'image_url', 'latitude', 'listing_date', 'location', 'longitude', 'manufacturer', 'mileage', 'model', 'price', 'source_dataset', 'transmission', 'vehicle_id', 'year']
Dataset B columns: ['body_type', 'cylinders', 'description', 'drive_type', 'exterior_color', 'fuel_type', 'image_url', 'latitude', 'listing_date', 'location', 'longitude', 'manufacturer', 'mileage', 'model', 'price', 'source_dataset', 'transmission', 'vehicle_id', 'year']
B1 candidates: 964002
B2 candidates: 77087


In [43]:
# Regenerate B1 and B2 lines with fixed format
print("Regenerating B1 lines...")
B1_lines_fixed, B1_pair_ids_fixed = prepare_blocking_for_ditto_fixed(B1_candidates, dataset_A, dataset_B)
print(f"B1: {len(B1_lines_fixed)} pairs")

print("\nRegenereating B2 lines...")
B2_lines_fixed, B2_pair_ids_fixed = prepare_blocking_for_ditto_fixed(B2_candidates, dataset_A, dataset_B)
print(f"B2: {len(B2_lines_fixed)} pairs")

# Compare old vs new format
print("\n=== OLD B1 line ===")
print(B1_lines[0])
print("\n=== NEW B1 line (fixed) ===")
print(B1_lines_fixed[0])
print("\n=== Sample TEST line ===")
print(test_all_lines[0])

Regenerating B1 lines...


Preparing candidates: 100%|██████████| 964002/964002 [00:13<00:00, 71812.69it/s]


B1: 964002 pairs

Regenereating B2 lines...


Preparing candidates: 100%|██████████| 77087/77087 [00:01<00:00, 72485.02it/s]

B2: 77087 pairs

=== OLD B1 line ===
COL year VAL 2014.0 COL manufacturer VAL Mazda COL model VAL Mazda5 COL price VAL 0 COL mileage VAL 60047.0 COL fuel VAL gas COL transmission VAL automatic	COL year VAL 2008 COL manufacturer VAL Mazda COL model VAL Mazda5 COL price VAL 4999.0 COL mileage VAL 130122.0 COL fuel VAL gasoline COL transmission VAL automatic	0

=== NEW B1 line (fixed) ===
COL year VAL 2014.0 COL manufacturer VAL Mazda COL model VAL Mazda5 COL price VAL 0 COL mileage VAL 60047.0	COL year VAL 2008 COL manufacturer VAL Mazda COL model VAL Mazda5 COL price VAL 4999.0 COL mileage VAL 130122.0	0

=== Sample TEST line ===
COL manufacturer VAL Nissan COL model VAL Maxima COL price VAL 3975 COL mileage VAL 147799.0	COL manufacturer VAL Chevrolet COL model VAL Equinox COL price VAL 23934.0 COL mileage VAL 5.0	0


In [44]:
# FINAL FIX: Match EXACTLY training format - NO year, just manufacturer, model, price, mileage

def prepare_blocking_for_ditto_final(candidates_df, dataset_A, dataset_B):
    """
    Create Ditto-format pairs from blocking candidates.
    Using EXACT same format as training data: manufacturer, model, price, mileage
    """
    dataset_A = dataset_A.copy()
    dataset_B = dataset_B.copy()
    
    A_id_col = 'vehicle_id'
    B_id_col = 'vehicle_id'
    
    dataset_A[A_id_col] = dataset_A[A_id_col].astype(str)
    dataset_B[B_id_col] = dataset_B[B_id_col].astype(str)
    
    A_dict = dataset_A.set_index(A_id_col).to_dict('index')
    B_dict = dataset_B.set_index(B_id_col).to_dict('index')
    
    lines = []
    pair_ids = []
    
    cand_A_col = 'id_A'
    cand_B_col = 'id_B'
    
    # EXACT same attributes as training: manufacturer, model, price, mileage (NO year!)
    attr_mapping = [
        ('manufacturer', 'manufacturer'),
        ('model', 'model'),
        ('price', 'price'),
        ('mileage', 'mileage'),
    ]
    
    for _, row in tqdm(candidates_df.iterrows(), total=len(candidates_df), desc="Preparing candidates"):
        id_A = str(row[cand_A_col])
        id_B = str(row[cand_B_col])
        
        if id_A not in A_dict or id_B not in B_dict:
            continue
        
        rec_A = A_dict[id_A]
        rec_B = B_dict[id_B]
        
        tokens_A = []
        tokens_B = []
        
        for ditto_attr, data_attr in attr_mapping:
            if data_attr in rec_A and pd.notna(rec_A[data_attr]):
                val = str(rec_A[data_attr]).strip()
                if val:
                    tokens_A.append(f"COL {ditto_attr} VAL {val}")
                    
        for ditto_attr, data_attr in attr_mapping:
            if data_attr in rec_B and pd.notna(rec_B[data_attr]):
                val = str(rec_B[data_attr]).strip()
                if val:
                    tokens_B.append(f"COL {ditto_attr} VAL {val}")
        
        rec1_str = ' '.join(tokens_A)
        rec2_str = ' '.join(tokens_B)
        
        # Dummy label
        line = f"{rec1_str}\t{rec2_str}\t0"
        lines.append(line)
        pair_ids.append((id_A, id_B))
    
    return lines, pair_ids

# Regenerate
print("Regenerating B1 lines with EXACT training format...")
B1_lines_final, B1_pair_ids_final = prepare_blocking_for_ditto_final(B1_candidates, dataset_A, dataset_B)
print(f"B1: {len(B1_lines_final)} pairs")

print("\nRegenerating B2 lines with EXACT training format...")
B2_lines_final, B2_pair_ids_final = prepare_blocking_for_ditto_final(B2_candidates, dataset_A, dataset_B)
print(f"B2: {len(B2_lines_final)} pairs")

# Verify format matches
print("\n=== NEW B1 line (final) ===")
print(B1_lines_final[0])
print("\n=== Sample TEST line (training format) ===")
print(test_all_lines[0])

Regenerating B1 lines with EXACT training format...


Preparing candidates: 100%|██████████| 964002/964002 [00:12<00:00, 76350.22it/s]


B1: 964002 pairs

Regenerating B2 lines with EXACT training format...


Preparing candidates: 100%|██████████| 77087/77087 [00:01<00:00, 75696.34it/s]

B2: 77087 pairs

=== NEW B1 line (final) ===
COL manufacturer VAL Mazda COL model VAL Mazda5 COL price VAL 0 COL mileage VAL 60047.0	COL manufacturer VAL Mazda COL model VAL Mazda5 COL price VAL 4999.0 COL mileage VAL 130122.0	0

=== Sample TEST line (training format) ===
COL manufacturer VAL Nissan COL model VAL Maxima COL price VAL 3975 COL mileage VAL 147799.0	COL manufacturer VAL Chevrolet COL model VAL Equinox COL price VAL 23934.0 COL mileage VAL 5.0	0


In [45]:
# Re-run inference with corrected format
import time

print("="*60)
print("RE-RUNNING INFERENCE WITH CORRECTED FORMAT")
print("="*60)

# B1 Inference
print("\nB1 Inference...")
B1_start = time.time()
B1_probs_corrected = predict_with_ditto(model, B1_lines_final, batch_size=256, lm=hp.lm, max_len=hp.max_len, device=hp.device)
B1_time = time.time() - B1_start
print(f"B1 Inference time: {B1_time:.2f} seconds")
print(f"B1 Predictions: {len(B1_probs_corrected)}")

# Check distribution
print(f"\nB1 Probability Distribution:")
print(f"  Min: {B1_probs_corrected.min():.4f}")
print(f"  Max: {B1_probs_corrected.max():.4f}")
print(f"  Mean: {B1_probs_corrected.mean():.4f}")
print(f"  Median: {np.median(B1_probs_corrected):.4f}")
print(f"  > 0.5: {(B1_probs_corrected > 0.5).sum()}")
print(f"  > 0.9: {(B1_probs_corrected > 0.9).sum()}")

RE-RUNNING INFERENCE WITH CORRECTED FORMAT

B1 Inference...


Predicting: 100%|██████████| 3766/3766 [04:14<00:00, 14.80it/s]

B1 Inference time: 255.72 seconds
B1 Predictions: 964002

B1 Probability Distribution:
  Min: 0.0000
  Max: 0.0059
  Mean: 0.0007
  Median: 0.0002
  > 0.5: 0
  > 0.9: 0


In [46]:
# Let's check if there's an issue with how training data was created
# Compare training positive pairs vs B1 pairs

print("=== Sample POSITIVE training pair (label=1) ===")
for line in test_all_lines:
    if line.endswith('\t1'):
        print(line[:500])
        break

print("\n=== Sample from B1 (with similar manufacturer) ===")
for line in B1_lines_final[:100]:
    if 'Mazda' in line:
        print(line[:500])
        break

# Check if there are any actual matches in B1 by looking at probabilities
print("\n=== B1 predictions stats ===")
print(f"Number of predictions > 0.3: {(B1_probs_corrected > 0.3).sum()}")
print(f"Number of predictions > 0.1: {(B1_probs_corrected > 0.1).sum()}")
print(f"Number of predictions > 0.01: {(B1_probs_corrected > 0.01).sum()}")
print(f"Number of predictions > 0.001: {(B1_probs_corrected > 0.001).sum()}")

=== Sample POSITIVE training pair (label=1) ===
COL year VAL 2013.0 COL manufacturer VAL Kia COL model VAL Soul + COL price VAL 8995 COL mileage VAL 105873.0	COL year VAL 2013.0 COL manufacturer VAL Kia COL model VAL Soul COL price VAL 9995.0 COL mileage VAL 105237.0	1

=== Sample from B1 (with similar manufacturer) ===
COL manufacturer VAL Mazda COL model VAL Mazda5 COL price VAL 0 COL mileage VAL 60047.0	COL manufacturer VAL Mazda COL model VAL Mazda5 COL price VAL 4999.0 COL mileage VAL 130122.0	0

=== B1 predictions stats ===
Number of predictions > 0.3: 0
Number of predictions > 0.1: 0
Number of predictions > 0.01: 0
Number of predictions > 0.001: 221172


In [47]:
# The training data has YEAR! Let me fix to use year
# Training format: COL year VAL ... COL manufacturer VAL ... COL model VAL ... COL price VAL ... COL mileage VAL ...

def prepare_blocking_for_ditto_with_year(candidates_df, dataset_A, dataset_B):
    """
    Create Ditto-format pairs from blocking candidates.
    Using EXACT same format as training data: year, manufacturer, model, price, mileage
    """
    dataset_A = dataset_A.copy()
    dataset_B = dataset_B.copy()
    
    A_id_col = 'vehicle_id'
    B_id_col = 'vehicle_id'
    
    dataset_A[A_id_col] = dataset_A[A_id_col].astype(str)
    dataset_B[B_id_col] = dataset_B[B_id_col].astype(str)
    
    A_dict = dataset_A.set_index(A_id_col).to_dict('index')
    B_dict = dataset_B.set_index(B_id_col).to_dict('index')
    
    lines = []
    pair_ids = []
    
    cand_A_col = 'id_A'
    cand_B_col = 'id_B'
    
    # EXACT same attributes as training: year, manufacturer, model, price, mileage
    attr_mapping = [
        ('year', 'year'),
        ('manufacturer', 'manufacturer'),
        ('model', 'model'),
        ('price', 'price'),
        ('mileage', 'mileage'),
    ]
    
    for _, row in tqdm(candidates_df.iterrows(), total=len(candidates_df), desc="Preparing candidates"):
        id_A = str(row[cand_A_col])
        id_B = str(row[cand_B_col])
        
        if id_A not in A_dict or id_B not in B_dict:
            continue
        
        rec_A = A_dict[id_A]
        rec_B = B_dict[id_B]
        
        tokens_A = []
        tokens_B = []
        
        for ditto_attr, data_attr in attr_mapping:
            if data_attr in rec_A and pd.notna(rec_A[data_attr]):
                val = str(rec_A[data_attr]).strip()
                if val:
                    tokens_A.append(f"COL {ditto_attr} VAL {val}")
                    
        for ditto_attr, data_attr in attr_mapping:
            if data_attr in rec_B and pd.notna(rec_B[data_attr]):
                val = str(rec_B[data_attr]).strip()
                if val:
                    tokens_B.append(f"COL {ditto_attr} VAL {val}")
        
        rec1_str = ' '.join(tokens_A)
        rec2_str = ' '.join(tokens_B)
        
        line = f"{rec1_str}\t{rec2_str}\t0"
        lines.append(line)
        pair_ids.append((id_A, id_B))
    
    return lines, pair_ids

# Regenerate WITH year
print("Regenerating B1 lines WITH year...")
B1_lines_v3, B1_pair_ids_v3 = prepare_blocking_for_ditto_with_year(B1_candidates, dataset_A, dataset_B)
print(f"B1: {len(B1_lines_v3)} pairs")

print("\nRegenerating B2 lines WITH year...")
B2_lines_v3, B2_pair_ids_v3 = prepare_blocking_for_ditto_with_year(B2_candidates, dataset_A, dataset_B)
print(f"B2: {len(B2_lines_v3)} pairs")

# Compare
print("\n=== Training positive sample ===")
print(test_all_lines[next(i for i,l in enumerate(test_all_lines) if l.endswith('\t1'))])
print("\n=== New B1 line (v3 with year) ===")
print(B1_lines_v3[0])

Regenerating B1 lines WITH year...


Preparing candidates: 100%|██████████| 964002/964002 [00:13<00:00, 72492.54it/s]


B1: 964002 pairs

Regenerating B2 lines WITH year...


Preparing candidates: 100%|██████████| 77087/77087 [00:01<00:00, 72109.84it/s]

B2: 77087 pairs

=== Training positive sample ===
COL year VAL 2013.0 COL manufacturer VAL Kia COL model VAL Soul + COL price VAL 8995 COL mileage VAL 105873.0	COL year VAL 2013.0 COL manufacturer VAL Kia COL model VAL Soul COL price VAL 9995.0 COL mileage VAL 105237.0	1

=== New B1 line (v3 with year) ===
COL year VAL 2014.0 COL manufacturer VAL Mazda COL model VAL Mazda5 COL price VAL 0 COL mileage VAL 60047.0	COL year VAL 2008 COL manufacturer VAL Mazda COL model VAL Mazda5 COL price VAL 4999.0 COL mileage VAL 130122.0	0


In [48]:
# Run inference with v3 (with year)
print("="*60)
print("B1 INFERENCE (with year)")
print("="*60)

B1_start_v3 = time.time()
B1_probs_v3 = predict_with_ditto(model, B1_lines_v3, batch_size=256, lm=hp.lm, max_len=hp.max_len, device=hp.device)
B1_time_v3 = time.time() - B1_start_v3
print(f"B1 Inference time: {B1_time_v3:.2f} seconds")

print(f"\nB1 Probability Distribution:")
print(f"  Min: {B1_probs_v3.min():.4f}")
print(f"  Max: {B1_probs_v3.max():.4f}")
print(f"  Mean: {B1_probs_v3.mean():.4f}")
print(f"  Median: {np.median(B1_probs_v3):.4f}")
print(f"  > 0.5: {(B1_probs_v3 > 0.5).sum()}")
print(f"  > 0.3: {(B1_probs_v3 > 0.3).sum()}")
print(f"  > 0.1: {(B1_probs_v3 > 0.1).sum()}")

B1 INFERENCE (with year)


Predicting: 100%|██████████| 3766/3766 [05:09<00:00, 12.17it/s]

B1 Inference time: 310.61 seconds

B1 Probability Distribution:
  Min: 0.9991
  Max: 0.9999
  Mean: 0.9999
  Median: 0.9999
  > 0.5: 964002
  > 0.3: 964002
  > 0.1: 964002


In [22]:
# Debug: Test the model on training examples directly to see what's happening

# Get some positive and negative examples from the training data
print("Testing model on known training examples...")
test_samples = []
test_sample_labels = []

# Get some negatives
for line in test_all_lines:
    if line.endswith('\t0') and len(test_samples) < 5:
        test_samples.append(line)
        test_sample_labels.append(0)
    elif line.endswith('\t1') and len(test_samples) < 10:
        test_samples.append(line)
        test_sample_labels.append(1)
    if len(test_samples) >= 10:
        break

# Predict
sample_probs = predict_with_ditto(model, test_samples, batch_size=16, lm=hp.lm, max_len=hp.max_len, device=hp.device)

print("\nPredictions on test samples:")
for i, (prob, label) in enumerate(zip(sample_probs, test_sample_labels)):
    print(f"  Sample {i+1}: True={label}, Pred prob={prob:.4f} ({'MATCH' if prob > 0.5 else 'NO MATCH'})")

Testing model on known training examples...


NameError: name 'test_all_lines' is not defined

In [50]:
# Compare character by character what's different

print("=== Test negative (correctly classified) ===")
test_neg = test_samples[0]
print(repr(test_neg))
print()

print("=== B1 sample (incorrectly classified as match) ===")
b1_sample = B1_lines_v3[0]
print(repr(b1_sample))
print()

# Check if there's a difference in tokenization
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Parse test sample
t_parts = test_neg.strip().split('\t')
print(f"Test sample parts: {len(t_parts)}")
print(f"Test left: {t_parts[0]}")
print(f"Test right: {t_parts[1]}")

# Parse B1 sample
b_parts = b1_sample.strip().split('\t')
print(f"\nB1 sample parts: {len(b_parts)}")
print(f"B1 left: {b_parts[0]}")
print(f"B1 right: {b_parts[1]}")

=== Test negative (correctly classified) ===
'COL manufacturer VAL Nissan COL model VAL Maxima COL price VAL 3975 COL mileage VAL 147799.0\tCOL manufacturer VAL Chevrolet COL model VAL Equinox COL price VAL 23934.0 COL mileage VAL 5.0\t0'

=== B1 sample (incorrectly classified as match) ===
'COL year VAL 2014.0 COL manufacturer VAL Mazda COL model VAL Mazda5 COL price VAL 0 COL mileage VAL 60047.0\tCOL year VAL 2008 COL manufacturer VAL Mazda COL model VAL Mazda5 COL price VAL 4999.0 COL mileage VAL 130122.0\t0'

Test sample parts: 3
Test left: COL manufacturer VAL Nissan COL model VAL Maxima COL price VAL 3975 COL mileage VAL 147799.0
Test right: COL manufacturer VAL Chevrolet COL model VAL Equinox COL price VAL 23934.0 COL mileage VAL 5.0

B1 sample parts: 3
B1 left: COL year VAL 2014.0 COL manufacturer VAL Mazda COL model VAL Mazda5 COL price VAL 0 COL mileage VAL 60047.0
B1 right: COL year VAL 2008 COL manufacturer VAL Mazda COL model VAL Mazda5 COL price VAL 4999.0 COL mileage VAL

In [51]:
# Check the actual training file content
with open(train_path, 'r', encoding='utf-8') as f:
    train_lines_actual = [f.readline().strip() for _ in range(5)]

print("=== Actual training file lines (first 5) ===")
for i, line in enumerate(train_lines_actual):
    print(f"{i+1}: {line[:150]}...")
    
print("\n=== Sample test_all_lines (first 5) ===")
for i, line in enumerate(test_all_lines[:5]):
    print(f"{i+1}: {line[:150]}...")

=== Actual training file lines (first 5) ===
1: COL manufacturer VAL Chevrolet COL model VAL Colorado Z71 4X4 Gas COL price VAL 39999 COL mileage VAL 13303.0	COL manufacturer VAL Chevrolet COL model...
2: COL manufacturer VAL Chevrolet COL model VAL Suburban COL price VAL 23999 COL mileage VAL 200754.0	COL manufacturer VAL Mazda COL model VAL Mazda6 COL...
3: COL manufacturer VAL Ford COL model VAL Explorer COL price VAL 0 COL mileage VAL 5507.0	COL manufacturer VAL Dodge COL model VAL Journey COL price VAL...
4: COL manufacturer VAL Jeep COL model VAL Wrangler COL price VAL 19999 COL mileage VAL 81879.0	COL manufacturer VAL Gmc COL model VAL Acadia COL price V...
5: COL year VAL 2016.0 COL manufacturer VAL Gmc COL model VAL Acadia COL price VAL 29999 COL mileage VAL 38264.0	COL year VAL 2016.0 COL manufacturer VAL...

=== Sample test_all_lines (first 5) ===
1: COL manufacturer VAL Nissan COL model VAL Maxima COL price VAL 3975 COL mileage VAL 147799.0	COL manufacturer VAL Chevrolet CO

In [52]:
# The issue might be simpler - let's check if B1 pairs are actually VERY similar
# (same manufacturer/model), which might legitimately look like matches to the model

# Let's check if B1 lines without year work better
B1_no_year = []
for line in B1_lines_v3[:100]:
    parts = line.split('\t')
    # Remove year from each part
    left = ' '.join([p for p in parts[0].split(' COL ') if not p.startswith('year')])
    right = ' '.join([p for p in parts[1].split(' COL ') if not p.startswith('year')])
    # Reconstruct with COL prefix
    left_fixed = 'COL ' + left.strip() if not left.startswith('COL') else left
    right_fixed = 'COL ' + right.strip() if not right.startswith('COL') else right
    B1_no_year.append(f"{left_fixed}\t{right_fixed}\t0")

# Actually - let me just use the version without year I already made
print("Testing B1_lines_final (without year) on first 100...")
sample_probs_noyear = predict_with_ditto(model, B1_lines_final[:100], batch_size=16, lm=hp.lm, max_len=hp.max_len, device=hp.device)

print(f"\nB1 (no year) first 100 distribution:")
print(f"  Min: {sample_probs_noyear.min():.4f}")
print(f"  Max: {sample_probs_noyear.max():.4f}")
print(f"  Mean: {sample_probs_noyear.mean():.4f}")
print(f"  > 0.5: {(sample_probs_noyear > 0.5).sum()}")

# Compare with year version
print("\nTesting B1_lines_v3 (with year) on first 100...")
sample_probs_year = predict_with_ditto(model, B1_lines_v3[:100], batch_size=16, lm=hp.lm, max_len=hp.max_len, device=hp.device)

print(f"\nB1 (with year) first 100 distribution:")
print(f"  Min: {sample_probs_year.min():.4f}")
print(f"  Max: {sample_probs_year.max():.4f}")
print(f"  Mean: {sample_probs_year.mean():.4f}")
print(f"  > 0.5: {(sample_probs_year > 0.5).sum()}")

Testing B1_lines_final (without year) on first 100...


Predicting: 100%|██████████| 7/7 [00:00<00:00, 33.06it/s]



B1 (no year) first 100 distribution:
  Min: 0.0001
  Max: 0.0030
  Mean: 0.0011
  > 0.5: 0

Testing B1_lines_v3 (with year) on first 100...


Predicting: 100%|██████████| 7/7 [00:00<00:00, 159.09it/s]


B1 (with year) first 100 distribution:
  Min: 0.9999
  Max: 0.9999
  Mean: 0.9999
  > 0.5: 100


In [53]:
# The issue: training data has INCONSISTENT format (some with year, some without)
# This caused the model to learn spurious correlations

# Let's check the distribution in training data
with open(train_path, 'r', encoding='utf-8') as f:
    all_train_lines = [l.strip() for l in f]
    
with_year = sum(1 for l in all_train_lines if 'COL year' in l)
without_year = len(all_train_lines) - with_year

print(f"Training lines with year: {with_year} ({100*with_year/len(all_train_lines):.1f}%)")
print(f"Training lines without year: {without_year} ({100*without_year/len(all_train_lines):.1f}%)")

# Check if there's a correlation between year presence and labels
with_year_labels = [int(l.split('\t')[-1]) for l in all_train_lines if 'COL year' in l]
without_year_labels = [int(l.split('\t')[-1]) for l in all_train_lines if 'COL year' not in l]

print(f"\nWith year - label=1: {sum(with_year_labels)} ({100*sum(with_year_labels)/len(with_year_labels):.1f}%)")
print(f"Without year - label=1: {sum(without_year_labels)} ({100*sum(without_year_labels)/len(without_year_labels):.1f}%)")

Training lines with year: 2724 (25.3%)
Training lines without year: 8064 (74.7%)

With year - label=1: 2724 (100.0%)
Without year - label=1: 0 (0.0%)


## FIX: Ricreare i dati di training con formato consistente

Abbiamo scoperto che il modello ha imparato una correlazione spuria:
- Righe CON "year" → 100% label=1 (match)
- Righe SENZA "year" → 100% label=0 (non-match)

Questo è dovuto a un bug nella creazione dei dati. Dobbiamo:
1. Ricreare i dati con formato consistente (sempre o mai year)
2. Ri-addestrare il modello
3. Ri-eseguire l'inference

In [6]:
# FIX: Recreate training/validation/test data with CONSISTENT format
# Always include: year, manufacturer, model, price, mileage (when available)

os.chdir(WORKSPACE)
train_df = pd.read_csv(os.path.join(GROUND_TRUTH_DIR, 'train.csv'))
valid_df = pd.read_csv(os.path.join(GROUND_TRUTH_DIR, 'validation.csv'))  # Fixed filename
test_df = pd.read_csv(os.path.join(GROUND_TRUTH_DIR, 'test.csv'))
os.chdir(DITTO_DIR)

def serialize_record_v2(row, prefix_cr=True):
    """
    Serialize a record to Ditto format with CONSISTENT attributes.
    Always includes: year, manufacturer, model, price, mileage
    """
    tokens = []
    
    if prefix_cr:
        # Craigslist record
        attrs = [
            ('year', 'year'),
            ('manufacturer', 'manufacturer_cr'),
            ('model', 'model_cr'),
            ('price', 'price_cr'),
            ('mileage', 'mileage_cr'),
        ]
    else:
        # Used Cars record  
        attrs = [
            ('year', 'year'),
            ('manufacturer', 'manufacturer_uc'),
            ('model', 'model_uc'),
            ('price', 'price_uc'),
            ('mileage', 'mileage_uc'),
        ]
    
    for attr_name, col_name in attrs:
        if col_name in row.index:
            val = row[col_name]
            if pd.notna(val):
                val_str = str(val).strip()
                if val_str:
                    tokens.append(f"COL {attr_name} VAL {val_str}")
    
    return ' '.join(tokens)


def convert_to_ditto_format_v2(df):
    """Convert to Ditto format with consistent attributes"""
    lines = []
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Converting"):
        rec1 = serialize_record_v2(row, prefix_cr=True)
        rec2 = serialize_record_v2(row, prefix_cr=False)
        label = int(row['label'])
        line = f"{rec1}\t{rec2}\t{label}"
        lines.append(line)
    return lines

print("Converting datasets with consistent format (always include year)...")
train_lines_v2 = convert_to_ditto_format_v2(train_df)
valid_lines_v2 = convert_to_ditto_format_v2(valid_df)
test_lines_v2 = convert_to_ditto_format_v2(test_df)

print(f"Train: {len(train_lines_v2)} pairs")
print(f"Valid: {len(valid_lines_v2)} pairs")
print(f"Test: {len(test_lines_v2)} pairs")

# Check for year consistency
with_year_new = sum(1 for l in train_lines_v2 if 'COL year' in l)
print(f"\nTrain lines with year: {with_year_new} / {len(train_lines_v2)}")

# Sample
print("\n=== Sample positive (label=1) ===")
pos = [l for l in train_lines_v2 if l.endswith('\t1')][0]
print(pos[:200])

print("\n=== Sample negative (label=0) ===")
neg = [l for l in train_lines_v2 if l.endswith('\t0')][0]
print(neg[:200])

Converting datasets with consistent format (always include year)...


Converting: 100%|██████████| 2313/2313 [00:00<00:00, 44860.42it/s]

Train: 10788 pairs
Valid: 2311 pairs
Test: 2313 pairs

Train lines with year: 2724 / 10788

=== Sample positive (label=1) ===
COL year VAL 2016.0 COL manufacturer VAL Gmc COL model VAL Acadia COL price VAL 29999 COL mileage VAL 38264.0	COL year VAL 2016.0 COL manufacturer VAL Gmc COL model VAL Acadia COL price VAL 27499.0 CO

=== Sample negative (label=0) ===
COL manufacturer VAL Chevrolet COL model VAL Colorado Z71 4X4 Gas COL price VAL 39999 COL mileage VAL 13303.0	COL manufacturer VAL Chevrolet COL model VAL Cruze COL price VAL 19990.0 COL mileage VAL 4


In [7]:
# Save the new consistent training files
train_path_v2 = os.path.join(DITTO_DATA_DIR, 'train.txt')
valid_path_v2 = os.path.join(DITTO_DATA_DIR, 'valid.txt')
test_path_v2 = os.path.join(DITTO_DATA_DIR, 'test.txt')

with open(train_path_v2, 'w', encoding='utf-8') as f:
    f.write('\n'.join(train_lines_v2))

with open(valid_path_v2, 'w', encoding='utf-8') as f:
    f.write('\n'.join(valid_lines_v2))

with open(test_path_v2, 'w', encoding='utf-8') as f:
    f.write('\n'.join(test_lines_v2))

print(f"\n✅ Saved consistent training files:")
print(f"   {train_path_v2}")
print(f"   {valid_path_v2}")
print(f"   {test_path_v2}")

# Verify no spurious correlation
with_year_new = sum(1 for l in train_lines_v2 if 'COL year' in l)
with_year_labels = [int(l.split('\t')[-1]) for l in train_lines_v2 if 'COL year' in l]
without_year_labels = [int(l.split('\t')[-1]) for l in train_lines_v2 if 'COL year' not in l]

print(f"\n✅ Verification - No spurious correlation:")
print(f"   With year ({with_year_new} lines): {sum(with_year_labels)} matches ({100*sum(with_year_labels)/len(with_year_labels):.1f}%)")
print(f"   Without year ({len(train_lines_v2)-with_year_new} lines): {sum(without_year_labels)} matches ({100*sum(without_year_labels)/len(without_year_labels):.1f}%)")



✅ Saved consistent training files:
   c:\Users\migli\HW6ID\FAIR-DA4ER\ditto\data\cars\train.txt
   c:\Users\migli\HW6ID\FAIR-DA4ER\ditto\data\cars\valid.txt
   c:\Users\migli\HW6ID\FAIR-DA4ER\ditto\data\cars\test.txt

✅ Verification - No spurious correlation:
   With year (2724 lines): 2724 matches (100.0%)
   Without year (8064 lines): 0 matches (0.0%)


In [8]:
# Check the training dataframe structure
print("Training dataframe columns:")
print(train_df.columns.tolist())
print(f"\nYear column null values: {train_df['year'].isna().sum()} / {len(train_df)}")

# Check if positive pairs have year
pos_df = train_df[train_df['label'] == 1]
neg_df = train_df[train_df['label'] == 0]

print(f"\nPositive pairs: {len(pos_df)} - year null: {pos_df['year'].isna().sum()}")
print(f"Negative pairs: {len(neg_df)} - year null: {neg_df['year'].isna().sum()}")

Training dataframe columns:
['vin', 'craigslist_id', 'used_cars_id', 'year', 'manufacturer_cr', 'manufacturer_uc', 'model_cr', 'model_uc', 'price_cr', 'price_uc', 'mileage_cr', 'mileage_uc', 'warnings', 'warning_details', 'label']

Year column null values: 8064 / 10788

Positive pairs: 2724 - year null: 0
Negative pairs: 8064 - year null: 8064


In [9]:
# Check dataset_A and dataset_B for year columns
print("Dataset A columns with year:")
print([c for c in dataset_A.columns if 'year' in c.lower()])
print(f"Dataset A year null: {dataset_A['year'].isna().sum()} / {len(dataset_A)}")

print("\nDataset B columns with year:")
print([c for c in dataset_B.columns if 'year' in c.lower()])
print(f"Dataset B year null: {dataset_B['year'].isna().sum()} / {len(dataset_B)}")

Dataset A columns with year:


NameError: name 'dataset_A' is not defined

In [10]:
# Load datasets to check year availability
os.chdir(WORKSPACE)
dataset_A = pd.read_csv(os.path.join(GROUND_TRUTH_DIR, 'dataset_A_no_vin.csv'))
dataset_B = pd.read_csv(os.path.join(GROUND_TRUTH_DIR, 'dataset_B_no_vin.csv'))

print(f"Dataset A: {len(dataset_A)} records")
print(f"Dataset A year null: {dataset_A['year'].isna().sum()} ({100*dataset_A['year'].isna().sum()/len(dataset_A):.1f}%)")

print(f"\nDataset B: {len(dataset_B)} records")
print(f"Dataset B year null: {dataset_B['year'].isna().sum()} ({100*dataset_B['year'].isna().sum()/len(dataset_B):.1f}%)")

# Check if train negatives could have year
print("\n--- Checking if negative pairs could have year ---")
negative_samples = train_df[train_df['label'] == 0].head(10)
for idx, row in negative_samples.iterrows():
    cr_id = str(row['craigslist_id'])
    uc_id = str(row['used_cars_id'])
    
    # Find in datasets
    rec_A = dataset_A[dataset_A['vehicle_id'].astype(str) == cr_id]
    rec_B = dataset_B[dataset_B['vehicle_id'].astype(str) == uc_id]
    
    if not rec_A.empty and not rec_B.empty:
        year_A = rec_A.iloc[0]['year'] if pd.notna(rec_A.iloc[0]['year']) else 'NULL'
        year_B = rec_B.iloc[0]['year'] if pd.notna(rec_B.iloc[0]['year']) else 'NULL'
        print(f"  Pair {idx}: A_year={year_A}, B_year={year_B}, train_year={row['year']}")
        if idx >= 5:
            break


Dataset A: 14492 records
Dataset A year null: 57 (0.4%)

Dataset B: 15375 records
Dataset B year null: 0 (0.0%)

--- Checking if negative pairs could have year ---
  Pair 0: A_year=2020.0, B_year=2019, train_year=nan
  Pair 1: A_year=2015.0, B_year=2020, train_year=nan
  Pair 2: A_year=2020.0, B_year=2020, train_year=nan
  Pair 3: A_year=2014.0, B_year=2020, train_year=nan
  Pair 5: A_year=2020.0, B_year=2021, train_year=nan


In [11]:
# FIX: Rebuild train/valid/test dataframes with year from original datasets
def rebuild_ground_truth_with_year(gt_df, dataset_A, dataset_B):
    """Rebuild ground truth with year from original datasets"""
    gt_fixed = gt_df.copy()
    
    # Create lookup dictionaries
    A_dict = dataset_A.set_index('vehicle_id').to_dict('index')
    B_dict = dataset_B.set_index('vehicle_id').to_dict('index')
    
    # Fix year column
    for idx, row in gt_fixed.iterrows():
        cr_id = str(row['craigslist_id'])
        uc_id = str(row['used_cars_id'])
        
        if cr_id in A_dict and uc_id in B_dict:
            rec_A = A_dict[cr_id]
            rec_B = B_dict[uc_id]
            
            # Use year from dataset_A (craigslist) or dataset_B if A is null
            year_A = rec_A.get('year')
            year_B = rec_B.get('year')
            
            if pd.notna(year_A):
                gt_fixed.at[idx, 'year'] = year_A
            elif pd.notna(year_B):
                gt_fixed.at[idx, 'year'] = year_B
    
    return gt_fixed

print("Rebuilding train/valid/test with year from original datasets...")
dataset_A['vehicle_id'] = dataset_A['vehicle_id'].astype(str)
dataset_B['vehicle_id'] = dataset_B['vehicle_id'].astype(str)
train_df['craigslist_id'] = train_df['craigslist_id'].astype(str)
train_df['used_cars_id'] = train_df['used_cars_id'].astype(str)
valid_df['craigslist_id'] = valid_df['craigslist_id'].astype(str)
valid_df['used_cars_id'] = valid_df['used_cars_id'].astype(str)
test_df['craigslist_id'] = test_df['craigslist_id'].astype(str)
test_df['used_cars_id'] = test_df['used_cars_id'].astype(str)

train_fixed = rebuild_ground_truth_with_year(train_df, dataset_A, dataset_B)
valid_fixed = rebuild_ground_truth_with_year(valid_df, dataset_A, dataset_B)
test_fixed = rebuild_ground_truth_with_year(test_df, dataset_A, dataset_B)

print(f"\n✅ Train fixed: year null = {train_fixed['year'].isna().sum()} / {len(train_fixed)}")
print(f"   Positives with year: {train_fixed[train_fixed['label']==1]['year'].notna().sum()} / {train_fixed['label'].sum()}")
print(f"   Negatives with year: {train_fixed[train_fixed['label']==0]['year'].notna().sum()} / {(1-train_fixed['label']).sum()}")

print(f"\n✅ Valid fixed: year null = {valid_fixed['year'].isna().sum()} / {len(valid_fixed)}")
print(f"✅ Test fixed: year null = {test_fixed['year'].isna().sum()} / {len(test_fixed)}")


Rebuilding train/valid/test with year from original datasets...

✅ Train fixed: year null = 0 / 10788
   Positives with year: 2724 / 2724
   Negatives with year: 8064 / 8064

✅ Valid fixed: year null = 0 / 2311
✅ Test fixed: year null = 0 / 2313


In [12]:
# Convert fixed dataframes to Ditto format
print("Converting fixed dataframes to Ditto format...")
train_lines_fixed = convert_to_ditto_format_v2(train_fixed)
valid_lines_fixed = convert_to_ditto_format_v2(valid_fixed)
test_lines_fixed = convert_to_ditto_format_v2(test_fixed)

# Save the fixed files
with open(train_path_v2, 'w', encoding='utf-8') as f:
    f.write('\n'.join(train_lines_fixed))

with open(valid_path_v2, 'w', encoding='utf-8') as f:
    f.write('\n'.join(valid_lines_fixed))

with open(test_path_v2, 'w', encoding='utf-8') as f:
    f.write('\n'.join(test_lines_fixed))

print(f"\n✅ Saved FIXED training files with year for all records")

# Verify NO spurious correlation now
with_year_count = sum(1 for l in train_lines_fixed if 'COL year' in l)
with_year_labels = [int(l.split('\t')[-1]) for l in train_lines_fixed if 'COL year' in l]
without_year_count = len(train_lines_fixed) - with_year_count

print(f"\n✅ VERIFICATION - Spurious correlation ELIMINATED:")
print(f"   Lines with year: {with_year_count} / {len(train_lines_fixed)} ({100*with_year_count/len(train_lines_fixed):.1f}%)")
if with_year_count > 0:
    print(f"     - Matches: {sum(with_year_labels)} ({100*sum(with_year_labels)/len(with_year_labels):.1f}%)")
    print(f"     - Non-matches: {len(with_year_labels)-sum(with_year_labels)} ({100*(len(with_year_labels)-sum(with_year_labels))/len(with_year_labels):.1f}%)")
if without_year_count > 0:
    without_year_labels = [int(l.split('\t')[-1]) for l in train_lines_fixed if 'COL year' not in l]
    print(f"   Lines without year: {without_year_count}")
    print(f"     - Matches: {sum(without_year_labels)} ({100*sum(without_year_labels)/len(without_year_labels):.1f}%)")
    print(f"     - Non-matches: {len(without_year_labels)-sum(without_year_labels)} ({100*(len(without_year_labels)-sum(without_year_labels))/len(without_year_labels):.1f}%)")


Converting fixed dataframes to Ditto format...


Converting: 100%|██████████| 2313/2313 [00:00<00:00, 44362.96it/s]


✅ Saved FIXED training files with year for all records

✅ VERIFICATION - Spurious correlation ELIMINATED:
   Lines with year: 10788 / 10788 (100.0%)
     - Matches: 2724 (25.3%)
     - Non-matches: 8064 (74.7%)


## 9. Summary and Save Results

In [38]:
# Summary
print("="*60)
print("DITTO MODEL - FINAL RESULTS")
print("="*60)

print(f"\nModel: {hp.lm}")
print(f"Training epochs: {hp.n_epochs}")
print(f"Training time: {training_time:.2f} seconds")

print("\n" + "-"*40)
print("B1-Ditto Results (threshold=0.5):")
print("-"*40)
b1_best = B1_results[1]  # threshold 0.5
print(f"  Precision: {b1_best['precision']:.4f}")
print(f"  Recall: {b1_best['recall']:.4f}")
print(f"  F1: {b1_best['f1']:.4f}")
print(f"  Inference time: {B1_inference_time:.2f} seconds")

print("\n" + "-"*40)
print("B2-Ditto Results (threshold=0.5):")
print("-"*40)
b2_best = B2_results[1]  # threshold 0.5
print(f"  Precision: {b2_best['precision']:.4f}")
print(f"  Recall: {b2_best['recall']:.4f}")
print(f"  F1: {b2_best['f1']:.4f}")
print(f"  Inference time: {B2_inference_time:.2f} seconds")

DITTO MODEL - FINAL RESULTS

Model: distilbert
Training epochs: 10
Training time: 84.07 seconds

----------------------------------------
B1-Ditto Results (threshold=0.5):
----------------------------------------
  Precision: 0.0127
  Recall: 0.9075
  F1: 0.0250
  Inference time: 406.29 seconds

----------------------------------------
B2-Ditto Results (threshold=0.5):
----------------------------------------
  Precision: 0.0676
  Recall: 0.9238
  F1: 0.1259
  Inference time: 34.33 seconds


In [40]:
# Save all results
os.chdir(WORKSPACE)

# Convert numpy types to Python native types for JSON serialization
def convert_to_native(obj):
    """Convert numpy types to Python native types"""
    if isinstance(obj, dict):
        return {k: convert_to_native(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_native(item) for item in obj]
    elif hasattr(obj, 'item'):  # numpy scalar
        return obj.item()
    else:
        return obj

results = {
    'model': hp.lm,
    'n_epochs': hp.n_epochs,
    'batch_size': hp.batch_size,
    'max_len': hp.max_len,
    'learning_rate': hp.lr,
    'device': hp.device,
    'training_time_seconds': training_time,
    'B1': {
        'inference_time_seconds': B1_inference_time,
        'num_candidates': len(B1_lines),
        'results_by_threshold': convert_to_native(B1_results)
    },
    'B2': {
        'inference_time_seconds': B2_inference_time,
        'num_candidates': len(B2_lines),
        'results_by_threshold': convert_to_native(B2_results)
    }
}

# Save JSON
with open(os.path.join(DITTO_RESULTS_DIR, 'ditto_metrics.json'), 'w') as f:
    json.dump(results, f, indent=2)

print(f"✅ Results saved to {os.path.join(DITTO_RESULTS_DIR, 'ditto_metrics.json')}")


✅ Results saved to c:\Users\migli\HW6ID\ditto_results\ditto_metrics.json


In [41]:
# Save predictions
B1_pred_df = pd.DataFrame({
    'craigslist_id': [p[0] for p in B1_pair_ids],
    'used_cars_id': [p[1] for p in B1_pair_ids],
    'probability': B1_probs,
    'predicted_match_05': (B1_probs >= 0.5).astype(int),
    'true_match': [1 if p in gt_set else 0 for p in B1_pair_ids]
})
B1_pred_df.to_csv(os.path.join(DITTO_RESULTS_DIR, 'ditto_predictions_B1.csv'), index=False)

B2_pred_df = pd.DataFrame({
    'craigslist_id': [p[0] for p in B2_pair_ids],
    'used_cars_id': [p[1] for p in B2_pair_ids],
    'probability': B2_probs,
    'predicted_match_05': (B2_probs >= 0.5).astype(int),
    'true_match': [1 if p in gt_set else 0 for p in B2_pair_ids]
})
B2_pred_df.to_csv(os.path.join(DITTO_RESULTS_DIR, 'ditto_predictions_B2.csv'), index=False)

print(f"Saved predictions to:")
print(f"  {os.path.join(DITTO_RESULTS_DIR, 'ditto_predictions_B1.csv')}")
print(f"  {os.path.join(DITTO_RESULTS_DIR, 'ditto_predictions_B2.csv')}")

Saved predictions to:
  c:\Users\migli\HW6ID\ditto_results\ditto_predictions_B1.csv
  c:\Users\migli\HW6ID\ditto_results\ditto_predictions_B2.csv


In [42]:
# Create threshold comparison table
comparison_data = []

for i, thresh in enumerate(thresholds):
    comparison_data.append({
        'Blocking': 'B1',
        'Threshold': thresh,
        'Precision': B1_results[i]['precision'],
        'Recall': B1_results[i]['recall'],
        'F1': B1_results[i]['f1'],
        'TP': B1_results[i]['tp'],
        'FP': B1_results[i]['fp']
    })
    comparison_data.append({
        'Blocking': 'B2',
        'Threshold': thresh,
        'Precision': B2_results[i]['precision'],
        'Recall': B2_results[i]['recall'],
        'F1': B2_results[i]['f1'],
        'TP': B2_results[i]['tp'],
        'FP': B2_results[i]['fp']
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df.to_csv(os.path.join(DITTO_RESULTS_DIR, 'threshold_comparison.csv'), index=False)

print("\nThreshold Comparison:")
print(comparison_df.to_string(index=False))


Threshold Comparison:
Blocking  Threshold  Precision   Recall       F1   TP     FP
      B1        0.3   0.010295 0.927057 0.020364 2186 210150
      B2        0.3   0.057628 0.942424 0.108614 2177  35600
      B1        0.5   0.012676 0.907549 0.025002 2140 166686
      B2        0.5   0.067557 0.923810 0.125907 2134  29454
      B1        0.7   0.015821 0.883800 0.031086 2084 129638
      B2        0.7   0.080366 0.900866 0.147568 2081  23813


In [43]:
print("\n" + "="*60)
print("TASK 4.G COMPLETED")
print("="*60)
print(f"All results saved to: {DITTO_RESULTS_DIR}")


TASK 4.G COMPLETED
All results saved to: c:\Users\migli\HW6ID\ditto_results
